# 환경설정

In [2]:
from dotenv import load_dotenv
load_dotenv()

from huggingface_hub import login
import os
login(token=os.getenv("HUGGINGFACE_API_KEY"))

In [3]:
import torch
print(torch.__version__)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

2.8.0+cu126
cuda


# 모델 불러오기

In [4]:
import os
import json
import json5
import re
from PIL import Image
import torch
from drugocr import extract_text
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
from transformers import AutoProcessor, AutoTokenizer, AutoModelForVision2Seq
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_8bit = False,
    load_in_4bit = True,
    lim_int8_threashold = 6.0,
    lim_int8_has_fp16_weight = False,
)


c:\Potenup\Drug-Detection-Chatbot\.venv\lib\site-packages\paddle\utils\cpp_extension\extension_utils.py:718: UserWarning: No ccache found. Please be aware that recompiling all source files may be required. You can download and install ccache from: https://github.com/ccache/ccache/blob/master/doc/INSTALL.md
  warnings.warn(warning_message)
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\user\.paddlex\official_models\PP-OCRv5_server_det`.
Creating model: ('korean_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\user\.paddlex\official_models\korean_PP-OCRv5_mobile_rec`.


In [5]:
# 양자화 모델 불러오기
model_name = "llava-hf/llava-v1.6-mistral-7b-hf"
processor = LlavaNextProcessor.from_pretrained(model_name, trust_remote_code = True)

model = AutoModelForVision2Seq.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    dtype=torch.bfloat16,   # torch_dtype -> dtype
)


Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
c:\Potenup\Drug-Detection-Chatbot\.venv\lib\site-packages\transformers\models\auto\modeling_auto.py:2242: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [22]:
# 입력 데이터 준비하기
# 이미지 파일을 base64로 인코딩
import base64

image_path = "C:\Potenup\Drug-Detection-Chatbot\data\medicine_00806.jpeg"
ocr_texts = extract_text(image_path)    # drugocr.py의 extract_text 함수 사용

with open(image_path, "rb") as image_file:
    image_data = base64.b64encode(image_file.read()).decode('utf-8')

system_prompt = """
당신은 약학 전문가입니다. 약학 전문가로서, 
사용자가 제공한 약봉지 이미지와 OCR 텍스트 데이터를 바탕으로 약봉지에 적힌 모든 정보를 정확하게 추출하는 역할을 합니다. 
사용자가 제공한 이미지와 OCR 텍스트 데이터를 분석하여, 약봉지에 적힌 모든 정보를 정확하게 추출해 안내하는게 중요합니다.
없으면 없다고 답하고, 예측하거나 텍스트 줄임없이 그대로 작성합니다.
성분/함량, 첨가물은 ',' 기준으로 한 단어로 인식합니다.
다음 JSON 형식에 맞춰 작성해주세요.

[출력형식]
{{
    '제품명' : "제품 이름, 없으면 없다고 작성할 것",
    '성분/함량' : "제품의 각 성분, 유효성분, 함량에 대한 튜플(성분명, 함량)들의 리스트, 없으면 없다고 작성할 것"
    '첨가물' : "제품의 첨가물/첨가제들의 리스트, 없으면 없다고 작성할 것",
    '제형/성상' : "제형, 성상, 없으면 없다고 작성할 것",
    'KPIC/ATC' : 'KPIC/A과C에 대한 리스트, 없으면 없다고 작성할 것",
    '구분' : "구분, 없으면 없다고 작성할 것"
    '효능' : "여러가지 효능/효과에 대한 리스트, 없으면 없다고 작성할 것",
    '용법' : "여러가지 용법에 대한 리스트, 없으면 없다고 작성할 것",
    '주의사항' : "주의사항 리스트, 없으면 없다고 작성할 것"
}}
"""

messages = [
    {
        "role": "system",
        "content":[{"type": "text", "text": system_prompt},]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": ocr_texts},
            {
                "type": "image",
                "source_type": "base64",
                "data": image_data,
                "mime_type": "image/jpeg",
            }
        ]
    }
]
    
# 입력 데이터 토크나이징하기
inputs = processor.apply_chat_template(
    messages,
    add_generation_prompt=True,
    #tokenizer=True,
    #return_dict=True,
    return_tensors="pt",
#).to(model.device, torch.float16)
).to(model.device)

output = model.generate(**inputs, max_new_tokens=50)
print(processor.decode(output[0], skip_special_tokens=True))


AttributeError: 'str' object has no attribute 'to'

In [5]:
# 2. system prompt 정의
# -------------------------
system_prompt = """
당신은 약학 전문가입니다. 약학 전문가로서,
사용자가 제공한 약봉지 이미지와 OCR 텍스트 데이터를 바탕으로
약봉지에 적힌 모든 정보를 정확하게 추출하는 역할을 합니다.

규칙:
- 제공된 데이터에 없는 항목은 반드시 "없음"이라고 작성할 것.
- 예측하지 말고, 텍스트를 줄이지 말고 그대로 작성할 것.
- 성분/함량, 첨가물은 ',' 기준으로 나누어 각각 항목으로 처리할 것.

출력은 아래 JSON 형식을 따르세요.

[출력형식]
{
    "제품명": "제품 이름 또는 없음",
    "성분/함량": [["성분명", "함량"], ["성분명", "함량"], ... ] 또는 없음,
    "첨가물": ["첨가제1", "첨가제2", ... ] 또는 없음,
    "제형/성상": "제형 또는 없음",
    "KPIC/ATC": ["KPIC 코드1", "ATC 코드2", ... ] 또는 없음,
    "구분": "구분 또는 없음",
    "효능": ["효능1", "효능2", ... ] 또는 없음,
    "용법": ["용법1", "용법2", ... ] 또는 없음,
    "주의사항": ["주의사항1", "주의사항2", ... ] 또는 없음
}
"""

# -------------------------
# 3. 데이터 준비
# -------------------------
image_path = r"C:\Potenup\Drug-Detection-Chatbot\data\medicine_00806.jpeg"
image = Image.open(image_path).convert("RGB")

# OCR 텍스트 (이미 추출한 함수 사용)
ocr_texts = extract_text(image_path)

# -------------------------
# 4. 메시지 구성
# -------------------------
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": system_prompt}]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": ocr_texts},
            {"type": "image"}   # 실제 이미지는 processor(...)에 따로 전달
        ]
    }
]

# -------------------------
# 5. Prompt 만들기
# -------------------------
prompt = processor.apply_chat_template(
    messages,
    add_generation_prompt=True
)

# -------------------------
# 6. 입력 토큰 변환 (텍스트+이미지)
# -------------------------
inputs = processor(
    text=prompt,
    images=image,
    return_tensors="pt",
    padding=True
).to(model.device)

# -------------------------
# 7. 모델 추론
# -------------------------
output = model.generate(
    **inputs,
    max_new_tokens=512
)

# -------------------------
# 8. 결과 디코딩
# -------------------------
decoded = processor.decode(output[0], skip_special_tokens=True)
print(decoded)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [11]:
# 2. system prompt 정의
# -------------------------
system_prompt = """
당신은 약학 전문가입니다. 약학 전문가로서,
사용자가 제공한 약봉지 이미지와 OCR 텍스트 데이터를 바탕으로
약봉지에 적힌 모든 정보를 정확하게 추출하는 역할을 합니다.

규칙:
- 제공된 데이터에 없는 항목은 반드시 "없음"이라고 작성할 것.
- 예측하지 말고, 텍스트를 줄이지 말고 그대로 작성할 것.
- 성분/함량, 첨가물은 ',' 기준으로 나누어 각각 항목으로 처리할 것.
- 출력은 반드시 JSON 형식만 출력하고, 그 외 텍스트(설명, 문장)는 절대 포함하지 말 것.

출력은 아래 JSON 형식을 따르세요.

[출력형식]
{
    "제품명": "제품 이름 또는 없음",
    "성분/함량": [["성분명", "함량"], ["성분명", "함량"], ... ] 또는 없음,
    "첨가물": ["첨가제1", "첨가제2", ... ] 또는 없음,
    "제형/성상": "제형 또는 없음",
    "KPIC/ATC": ["KPIC 코드1", "ATC 코드2", ... ] 또는 없음,
    "구분": "구분 또는 없음",
    "효능": ["효능1", "효능2", ... ] 또는 없음,
    "용법": ["용법1", "용법2", ... ] 또는 없음,
    "주의사항": ["주의사항1", "주의사항2", ... ] 또는 없음
}
"""

# -------------------------
# 3. 데이터 준비
# -------------------------
image_path = r"C:\Potenup\Drug-Detection-Chatbot\data\medicine_00806.jpeg"
image = Image.open(image_path).convert("RGB")

# OCR 텍스트 (이미 추출한 함수 사용)
ocr_texts = extract_text(image_path)

In [12]:
# 4. 메시지 구성
# -------------------------
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": system_prompt}]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": ocr_texts},
            {"type": "image"}   # 실제 이미지는 processor(...)에 따로 전달
        ]
    }
]

# -------------------------
# 5. Prompt 만들기
# -------------------------
prompt = processor.apply_chat_template(
    messages,
    add_generation_prompt=True
)


In [13]:
# 6. 입력 토큰 변환 (텍스트+이미지)
# -------------------------
inputs = processor(
    text=prompt,
    images=image,
    return_tensors="pt",
    padding=True
).to(model.device)

In [14]:
# -------------------------
# 7. 모델 추론
# -------------------------
output = model.generate(**inputs, max_new_tokens=50)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
